# Hybrid Quantum-Classical Medical Imaging on Colab

This notebook sets up the environment to run the QML medical imaging project directly from GitHub using Google Colab.

## key Features
- Clones the repository from GitHub.
- Installs dependencies.
- Mounts Google Drive for persistent data and output storage.
- Runs training/inference scripts.

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define paths
DATASET_PATH = '/content/drive/MyDrive/medical_imaging_data/brain_tumor_dataset' # @param {type:"string"}
OUTPUT_PATH = '/content/drive/MyDrive/qml_medical_imaging_outputs' # @param {type:"string"}

In [ ]:
# 2. Clone Repository
!git clone https://github.com/DreamX55/QML-for-Medical-Imaging.git
%cd QML-for-Medical-Imaging
!pip install -r requirements.txt

In [ ]:
# 3. Symlink Data and Outputs
# This allows the code to run without changing paths, while storing heavy files in Drive
import os

if not os.path.exists('data/brain_mri'):
    os.makedirs('data', exist_ok=True)
    os.symlink(DATASET_PATH, 'data/brain_mri')
    print(f"Linked dataset from {DATASET_PATH} to data/brain_mri")

if not os.path.exists('outputs'):
    os.symlink(OUTPUT_PATH, 'outputs')
    print(f"Linked outputs to {OUTPUT_PATH}")

In [ ]:
# 4. Run Training
!python main.py --config configs/hybrid_model.yaml